## Проверяем размеры

In [3]:
!head ./trainGraph/part-r-00000

1424	{(846,0),(1691,256),(2338,0),(4950,0),(7238,0),(7433,0),(7579,0),(11194,0),(19675,1024),(20969,8),(25735,0),(26263,0),(35936,0),(43730,1024),(46800,0),(48593,0),(53261,0),(58957,0),(66826,1024),(66908,0),(74521,0),(88314,0),(94583,0),(95533,0),(102053,0),(104554,0),(118467,0),(122267,0),(133294,0),(134168,0),(141617,0),(145693,0),(146901,0),(153592,0),(158038,0),(159507,0),(170944,256),(172171,0),(178749,0),(180034,0),(189202,0),(190509,0),(198871,0),(201924,0),(203249,0),(205763,0),(206270,0),(208670,0),(210363,0),(210852,0),(211987,0),(217279,0),(220930,0),(225781,0),(226374,16384),(239469,1024),(243230,0),(248387,0),(270779,0),(271107,1024),(275823,0),(281249,0),(285389,0),(292914,0),(295893,0),(311622,0),(313504,512),(319401,0),(332230,1024),(334133,0),(339120,16384),(341441,0),(349816,0),(350522,0),(360253,0),(361671,0),(364898,0),(374688,0),(375718,0),(377728,0),(384448,0),(388003,32),(389077,0),(394646,0),(401375,0),(409151,0),(409313,0),(417615,0),(420181,512),(428163,0),(

In [5]:
!head ./coreDemography/part-r-00000

16460783	1182885174073	486	2	10414533690	1078547	85
16467391	1176953226317	4669	2	10414533690	1384327	85
16467889	1169816093060	6861	2	10414533690	33438	
16468013	1172074147460	5360	2	10415971874	4000749	1
16471027	1182034019343	6311	2	10414533690	3047069	11
16474162	1173115608560	5567	2	10414533690	3868434	36
16476386	1151057658677	4351	2	10414533690	3385314	11
16481067	1177305838543	3831	2	10414533690	1112209	13
16485965	1178801047573	-2369	2	10415971874	2225871	96
16486027	1163679273840	5864	2	10397571399	3638536	89


In [6]:
!ls -lh ./trainGraph/part-r-00000

-rw-rw-r-- 1 natalya.pritykovskaya natalya.pritykovskaya 27M Mar 23 18:46 ./trainGraph/part-r-00000


In [7]:
!ls -lh ./trainGraph/

total 180M
-rw-rw-r-- 1 natalya.pritykovskaya natalya.pritykovskaya  27M Mar 23 18:46 part-r-00000
-rw-rw-r-- 1 natalya.pritykovskaya natalya.pritykovskaya 9.4M Mar 23 18:39 part-r-00000.gz
-rw-rw-r-- 1 natalya.pritykovskaya natalya.pritykovskaya 9.5M Mar 23 18:39 part-r-00001.gz
-rw-rw-r-- 1 natalya.pritykovskaya natalya.pritykovskaya 9.7M Mar 23 18:39 part-r-00002.gz
-rw-rw-r-- 1 natalya.pritykovskaya natalya.pritykovskaya 9.5M Mar 23 18:40 part-r-00003.gz
-rw-rw-r-- 1 natalya.pritykovskaya natalya.pritykovskaya 9.8M Mar 23 18:40 part-r-00004.gz
-rw-rw-r-- 1 natalya.pritykovskaya natalya.pritykovskaya 9.2M Mar 23 18:40 part-r-00005.gz
-rw-rw-r-- 1 natalya.pritykovskaya natalya.pritykovskaya  10M Mar 23 18:40 part-r-00006.gz
-rw-rw-r-- 1 natalya.pritykovskaya natalya.pritykovskaya 9.7M Mar 23 18:40 part-r-00007.gz
-rw-rw-r-- 1 natalya.pritykovskaya natalya.pritykovskaya 9.6M Mar 23 18:40 part-r-00008.gz
-rw-rw-r-- 1 natalya.pritykovskaya natalya.pritykovskaya 9.3M Mar 23 18:40 part-r-

__27 * 16 = 432 мегабайта - piece of cake!__

### Применительно к нашей задаче

* Первый прогноз можно построить по счетчику общих друзей
* Выданный граф не симетричный


<img src="pics/asymmetrical_graph.png" width=700/>

<img src="pics/asymmetrical_graph_2.png" width=700/>

## Попытка №1

Рекомендуем в качестве друзей тех, у кого много общих друзей с выбранным пользователем. Для этого втягиваем в память словарь вида [таргет пользователь -> его друзья], и его же в "развернутом виде" [друг ->таргет пользователи, у которых он в друзьях]

### Готовим окружение

In [8]:
import csv, gzip, os, random, numpy, math, timeit, copy, psutil

from collections import defaultdict

# Пути к данным
dataPath = "./"
graphPath = os.path.join(dataPath, "trainGraph")
demographyPath = os.path.join(dataPath, "coreDemography")
csv.field_size_limit(200000)

# Утилита для замера памяти
def memory_usage():
    process = psutil.Process(os.getpid())
    print("Memory usage {0!s} mb.".format(process.memory_info()[0] / 1024 / 1024))

## Попытка №1 - читаем данные

In [9]:
%%time
mineFriends = defaultdict(list)
friendsOfMine = defaultdict(list)

# Итерируемся по файлам в папке
for file in [f for f in os.listdir(graphPath) if f.endswith("gz")]:
    csv_reader = csv.reader(gzip.open(os.path.join(graphPath, file), "rt"), delimiter='\t')
    # А теперь по строкам в файле
    for line in csv_reader:
        user = int(line[0]) 
        # Разбираем идшки и маски друзей
        for friendship in line[1].replace("{(", "").replace(")}", "").split("),("):
            parts = friendship.split(",")
            friend = int(parts[0])
            mineFriends[user] += [friend]
            friendsOfMine[friend] += [user]

CPU times: user 2min 15s, sys: 10.3 s, total: 2min 26s
Wall time: 2min 27s


<img src="pics/reversed_dict.png" width=700/>

## Попытка №1 - тревожные сигналы

1.5 минуты на загрузку 432 мегабайт явно долговато, а что по памяти?

In [10]:
memory_usage()

Memory usage 3462.91796875 mb.


Итого два словарика в памяти __уже__ заняли больше трех гигабайт и дальше ворочать их будет неудобно. Но поробуем...

## Попытка №1 - пробуем расчет

In [12]:
%%time
friendsOfFriends = defaultdict(lambda: defaultdict(int))

for (user, friends) in mineFriends.items():
    for friend in friends:
        for theirFriend in friendsOfMine[friend]:
            friendsOfFriends[user][theirFriend] += 1

KeyboardInterrupt: 

## Попытка №1 - работа над ошибками

In [7]:
memory_usage()

Memory usage 64.0078125 mb.


### Применительно к нашей задаче

* Оценим объем данных
* Не забудем о накладных расходах!

## Разбор полетов - исследуем данные

Подумаем о том, как использовать память эффективнее. Для начала давайте попробуем прикинуть базовые показатели графа с которым работаем.

In [1]:
import csv, gzip, os, random, numpy, math, timeit, copy, psutil

from collections import defaultdict

# Пути к данным
dataPath = "./"
graphPath = os.path.join(dataPath, "trainGraph")
demographyPath = os.path.join(dataPath, "CoreDemography")
csv.field_size_limit(200000)

# Утилитка для замера памяти
def memory_usage():
    process = psutil.Process(os.getpid())
    print("Memory usage {0!s} mb.".format(process.memory_info()[0] / 1024 / 1024))

In [2]:
numUsersFrom = 0
numLinks = 0
maxUserIdFrom = 0
maxUserIdTo = 0
allIds = set()

for file in [f for f in os.listdir(graphPath) if f.endswith("gz")]:
    csv_reader = csv.reader(gzip.open(os.path.join(graphPath, file), "rt"), delimiter='\t')
    for line in csv_reader:
        user = int(line[0])
        numUsersFrom += 1
        maxUserIdFrom = max(user, maxUserIdFrom)
        allIds.add(user)
        for friendship in line[1].replace("{(", "").replace(")}", "").split("),("):
            parts = friendship.split(",")
            friend = int(parts[0])
            allIds.add(friend)
            numLinks += 1
            maxUserIdTo = max(maxUserIdTo, friend)
            
print("Num users from {0}, max {1}, num total {2}, max {3}, num links {4}".format(
        numUsersFrom, maxUserIdFrom, len(allIds), maxUserIdTo, numLinks))

Num users from 107474, max 16619036, num total 12417564, max 16619131, num links 36192484


### Оценим объем нужной памяти

На одну связь надо 2 айдишки, которые 4-х байтовый инт

In [3]:
numLinks * (4 + 4) / 1024 / 1024

276.1267395019531

__Почему вместо 276Мб мы получили сразу 3Гб? Почему??!!__

## Разбор полетов - смотрим в код python'а

```c
#define PyDict_MINSIZE 8
typedef struct _dictobject PyDictObject;
struct _dictobject {
  ...
  PyDictEntry *ma_table;
  ...
};

typedef struct {
  Py_ssize_t me_hash;
  PyObject *me_key;
  PyObject *me_value;
} PyDictEntry;
```

## Хэш-таблица с открытой адресацией

<img src="https://upload.wikimedia.org/wikipedia/commons/b/bf/Hash_table_5_0_1_1_1_1_0_SP.svg" width="800" />

## Разбор полетов - считаем накладные расходы

* На запись
  * +8 байт на указатель на запись (PyDictEntry)
  * +4 байта на хэш
  * +8*2 байтов на указатели на ключ/значение
* +30% свободных ячеек
* х2 свободных ячеек при копировании

In [4]:
(numLinks * ((4 + 4) + (8 + 4 + 16)) + 1.3 * 2 * 8 * numLinks) / 1024 / 1024

1960.4998504638672

Становится яснее!

## А ведь есть еще вопрос локальности данных!


<center>
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/e/e5/Von_Neumann_Architecture.svg/800px-Von_Neumann_Architecture.svg.png" width="600"/>
</center>


<center>
<img src="https://s3.amazonaws.com/media-p.slid.es/uploads/71982/images/2310246/figure04-caching-pyramid.png" width="600"/>
</center>


| Source   | Latency       |
| ---------|:-------------:|
| L1 Cache |  1-2 ns       |
| L2 Cache |  3-5 ns       |
| L3 Cache |  10-40 ns     |
| DRAM     |  60-100 ns    |



## Боксинг
```c
#define PyObject_HEAD                   \
    _PyObject_HEAD_EXTRA                \
    Py_ssize_t ob_refcnt;               \
    struct _typeobject *ob_type;

typedef struct {
    PyObject_HEAD
    long ob_ival;
} PyIntObject;
```

Кладя инт в "коробку" получаем 20 байт вместо 4-х, итого:

In [8]:
(numLinks * ((20 + 20) + (8 + 4 + 16)) + 1.3 * 2 * 8 * numLinks) / 1024 / 1024

3065.0068084716795

## Все ли проблемы вскрыли?

__Проблема заполнености__ - общие друзья есть у гораздо большего числа пар пользователей, чем пар пользователей, напрямую связанных дружбой

In [9]:
numUsersFrom * numUsersFrom * 4 / 1024 / 1024

44062.273696899414

### Применительно к нашей задаче

* Считаем общих друзей только там, где это нужно (входят в тестовое множество)

## Попытка №2

* Выбираем структуру данных
* Втягиваем правильно
* Считаем общих друзей для юзеров из тестового множества
* Профит!

## Выбираем структуры данных

### Словари

* Легко пользоватся.
* Но _ОЧЕНЬ_ много накладных расходов: указатель на запись, указатель на ключ/значение, объектная обертка для ключа и значения.
* Итого: на пару инт-инт _накладных_ расходов 48 байт.
* Данные размазаны по памяти и фрагментированны - кеш процессора резко терят эффективность.

### Списки

* Если ключ инт - можно использовать вместо словаря, как список списков
* НО! Список в питоне это массив указателей, а значит накладные расходы на указатель и объектную обертку ключа. остаются - 20 байт накладных расходов на запись :(.
* Проблема с потерей эффективности кеша сохраняется.

### Массивы numpy

* На пару инт-инт нужно всего 4 байта.
* Кеш процессора работает.
* Но нужно чтобы ключи шли последовательно (к счастью, в наших даных это так почти так: максимальный ид 16619131 при общем числе 12417564).
* Для того чтобы заполнить массив нужно знать его размер изначально.
* Если делать двумерный массив 100к на 16м - будет очень, очень, очень больно...

### Спарс матрицы scipy

* Предназначены для хранения двухмерных матриц с очень большим количеством нулей.
* Эффективные варианты хранят в виде нескольких массивов numpy.
* COO - три массива одинаковой длинны моделируют списов троек [(i,j,v)]. Просто строить, легко интегрировать с пандой.
* CSR - один маленьки массив с индексом начала для каждого ряда (монотонно растет), и два массива равно длинны с ид колонки и значением: [{i : [(j,v)]}]. Более компактна в памяти, быстрые опреации.
* CSC - тоже самое что CSR, но индексированны колонки.

#### CSR matrix
<img src="http://hamukazu.com/wp-content/uploads/2014/12/csr_matrix.png" width="800"/>

#### CSC matrix
<img src="http://hamukazu.com/wp-content/uploads/2014/12/csc_matrix2.png" width="800"/>

## Попытка № 2 - наш выбор

* Инициализруем граф в виде COO матрицы, затем переводим в CSR.
* Развернутый граф получаем с помощью транспонирования.
* Общих друзей считаем умножая матрицы.

#### Умножение матриц? WTF?


<img src="pics/matrix_mult.png" width=700/>

## Попытка №2 - готовим окружение

In [10]:
# Почистим не нужное и оставим константы чтоб потом не пресчитывать
del allIds
maxUserId = 16619131
numLinks = 38031656

In [11]:
import numpy
# Работа с матрицами (подсчет общих друзей реализован как умножение матрицы графа 
# самое на себя)
import scipy
from scipy.sparse import coo_matrix, csr_matrix

## Попытка №2 - читаем матрицу

In [13]:
fromUser = numpy.zeros((numLinks), dtype=numpy.int32) 
toUser = numpy.zeros((numLinks), dtype=numpy.int32) 
data = numpy.ones((numLinks), dtype=numpy.int16) 

current = 0

# Итерируемся по файлам в папке... 
for file in [f for f in os.listdir(graphPath) if f.endswith("gz")]:
    csv_reader = csv.reader(gzip.open(os.path.join(graphPath, file), "rt"), delimiter='\t')

    for line in csv_reader:
        user = int(line[0]) 
        maxUserId = max(user, maxUserId)
        # Разбираем идшки и маски друзей
        for friendship in line[1].replace("{(", "").replace(")}", "").split("),("):
            parts = friendship.split(",")
            # Записываем связь в массивы и двигаем указатель
            fromUser[current] = user
            friend = int(parts[0])
            toUser[current] = friend
            maxUserId = max(friend, maxUserId)
            current += 1
            

Собственно из массивов создаем COO матрицу

In [14]:
fullMatrix = coo_matrix(
    (data, (fromUser, toUser)),
    shape=(maxUserId + 1, maxUserId + 1)).tocsr()

В матрице три массива, соответственно легко померять сколько памяти она занимает

In [15]:
(fullMatrix.data.nbytes + fullMatrix.indptr.nbytes + fullMatrix.indices.nbytes) / (1024 * 1024)

270.49202156066895

Теперь аккуратно вычистим ненужное

In [16]:
del fromUser
del toUser
del data  

## Попытка №2 - считаем общих друзей для пользователей, по которым необходимо сформировать предсказание

In [17]:
targetUsers = set()
csv_reader = csv.reader(open(os.path.join(dataPath, "prediction.csv")))
for line in csv_reader: 
    targetUsers.add(int(line[0]))

In [18]:
targetUsersMatrix = copy.deepcopy(fullMatrix)

for i in range(maxUserId + 1):
    if i not in targetUsers:
        ptr = targetUsersMatrix.indptr[i]
        ptr_next = targetUsersMatrix.indptr[i+1]
        if ptr != ptr_next:
            targetUsersMatrix.data[ptr:ptr_next].fill(0)

In [19]:
targetUsersMatrix.eliminate_zeros()

In [20]:
%%time
commonFriends = targetUsersMatrix.dot(fullMatrix)

CPU times: user 35.7 s, sys: 1.18 s, total: 36.9 s
Wall time: 37.1 s


In [21]:
memory_usage()

Memory usage 1579.85546875 mb.


In [22]:
len(targetUsers)

7088

## Что будет, если мы попробуем посчитать общих друзей для всех пользователей

In [25]:
%%time
commonFriends = fullMatrix.dot(fullMatrix)

CPU times: user 12min 31s, sys: 1min 45s, total: 14min 17s
Wall time: 14min 40s


In [26]:
memory_usage()

Memory usage 27438.48046875 mb.




## Семинар

Для targetUsers посчитать оценку  возраста, как средние от возрастов друзей

In [23]:
demographyPath

'./CoreDemography'

In [24]:
birthDates = numpy.zeros(maxUserId + 1, dtype=numpy.int32)

# Iterate all files in demography path
for file in [f for f in os.listdir(demographyPath) if f.endswith("gz")]:
    csvinput = gzip.open(os.path.join(demographyPath, file), "rt")
    csv_reader = csv.reader(csvinput, delimiter='\t')
   
    # Extract birth date from each line
    for line in csv_reader:
        user = int(line[0])
        birthDates[user] = int(line[2]) if line[2] != '' else 0

In [25]:
print ("Memory used by demography: %s (mb)" % (birthDates.nbytes / (1024 * 1024)))

Memory used by demography: 63.39695739746094 (mb)
